In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import geojson
import firedrake
import icepack, icepack.plot, icepack.models

# Helheim Glacier

In this demo we'll look at Helheim Glacier in southeast Greenland.
Helheim is one of the bigger glaciers draining the Greenland Ice Sheet by mass flux along with Jakobshavn, Kangerdlugssuaq, Petermann, and the Northeast Greenland Ice Stream.
Much of the procedure for working with real data should be familiar from the demo on [inverse problems](https://icepack.github.io/icepack.demo.04-ice-shelf-inverse.html), where we inferred the fluidity of the Larsen C Ice Shelf in the Antarctic Peninsula.
Here we'll use many of the same techniques, only instead of inferring the fluidity, we'll look at the friction coefficient for glacier sliding over the underlying bedrock.
In the following, we'll assume that ice flow is by Weertman sliding:

$$\tau_b = -C|u|^{\frac{1}{m} - 1}u,$$

where $\tau_b$ is the basal shear stress, $u$ is the sliding velocity, $m \approx 3$ is the friction exponent, and $C$ is the friction coefficient.
First, we'll estimate the friction coefficient using a 2D, depth-averaged model.
Then we'll use this solution as an initial guess using the 3D hybrid model that we showed in [this demo](https://icepack.github.io/icepack.demo.05-hybrid-ice-stream.html) for a synthetic ice stream.

### Input data

Loading in the mesh and input data should be mostly familiar from the previous demos on Larsen C.
We'll be using different data sets since we're looking at Greenland rather than Antarctica this time around.

In [ ]:
outline_filename = icepack.datasets.fetch_helheim_outline()
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)

geometry = icepack.meshing.collection_to_geo(outline, lcar=5e3)
with open('helheim.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o helheim.msh helheim.geo

In [ ]:
mesh = firedrake.Mesh('helheim.msh')

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes)
axes.legend()

In [ ]:
thickness_filename = icepack.datasets.fetch_bedmachine_greenland()
thickness = rasterio.open('netcdf:' + thickness_filename + ':thickness', 'r')
surface = rasterio.open('netcdf:' + thickness_filename + ':surface', 'r')

Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
h0 = icepack.interpolate(thickness, Q)
s0 = icepack.interpolate(surface, Q)

from firedrake import inner, grad, dx
def smooth(q0, α):
    q = q0.copy(deepcopy=True)
    J = 0.5 * ((q - q0)**2 + α**2 * inner(grad(q), grad(q))) * dx
    F = firedrake.derivative(J, q)
    firedrake.solve(F == 0, q)
    return q

α = firedrake.Constant(2e3)
h = smooth(h0, α)
s = smooth(s0, α)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(h, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(s, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
velocity_filenames = icepack.datasets.fetch_measures_greenland()
vx_filename = [f for f in velocity_filenames if 'vx' in f][0]
vy_filename = [f for f in velocity_filenames if 'vy' in f][0]
vx = rasterio.open(vx_filename, 'r')
vy = rasterio.open(vy_filename, 'r')

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
u_obs = icepack.interpolate((vx, vy), V)

Now that we've loaded in the ice thickness and surface elevation, we'll calculate the gravitational driving stress

$$\tau_d = -\rho_Igh\nabla s.$$

The gravitational driving stress is in balance with internal viscous stresses and with basal stress.
By knowing roughly what the magnitudes of the driving stress and the velocity are, we can get a better idea of what a reasonable starting value of the friction coefficient should be.

In [ ]:
from firedrake import grad
from icepack.constants import ice_density as ρ_I, gravity as g
τ = firedrake.project(-ρ_I * g * h * grad(s), V)

In [ ]:
import numpy as np
fig, axes = icepack.plot.subplots()
levels = np.linspace(0, .5, 51)
contours = icepack.plot.tricontourf(τ, levels=levels, extend='max', axes=axes)
fig.colorbar(contours, label='megapascals')

To initialize the inverse problem, we'll assume that the basal shear stress takes up half of the driving stress.
This is a totally ad hoc assumption and it's not obvious a priori that this is going to give us reasonable values of the ice velocity.
Nonetheless, it happens to work well enough as a starting point.
We'll also need to re-parameterize the basal shear stress in terms of some auxiliary variable $\theta$ in order to guarantee that the friction coefficient is strictly positive.
In the following, we'll use

$$\tau_b = -\frac{\tau_0}{u_0^{1/m}}e^{-\phi}|u|^{\frac{1}{m} - 1}u$$

where $\tau_0$ and $u_0$ are the average magnitudes of the driving stress and speed respectively.

In [ ]:
from firedrake import exp, ln, sqrt, assemble
from icepack.constants import weertman_sliding_law as m
import icepack.models.friction

u = u_obs.copy(deepcopy=True)
speed = sqrt(inner(u, u))
stress = sqrt(inner(τ, τ))

area = assemble(firedrake.Constant(1) * dx(mesh))
speed_avg = assemble(speed * dx) / area
stress_avg = assemble(stress * dx) / area

print('Average speed:  {} meters / year'.format(speed_avg))
print('Average stress: {} kilopascals'.format(1e3 * stress_avg))

fraction = 0.5
C = fraction * stress / speed**(1/m)
ϕ = firedrake.interpolate(-ln(speed_avg**(1/m) * C / stress_avg), Q)

def bed_friction(u, ϕ):
    C = stress_avg / speed_avg**(1/m) * exp(-ϕ)
    return icepack.models.friction.bed_friction(u, C)

For now, we'll assume that the ice is at a constant -13C.
We'll revisit this assumption using a heat flow model later.

In [ ]:
T = firedrake.Constant(260.)
A = icepack.rate_factor(T)

In [ ]:
ice_stream = icepack.models.IceStream(friction=bed_friction)
opts = {'dirichlet_ids': [1, 2, 4, 5, 6], 'tol': 1e-6}
u = ice_stream.diagnostic_solve(u0=u_obs, h=h, s=s, A=A, ϕ=ϕ, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u, axes=axes)
fig.colorbar(contours, label='meters / year')